## 七月在线机器学习实训营三期第七周(机器学习-模型调优与融合)考试
#### 考试说明:
- 起止时间：请同学在2018年3月23日至3月25日期间完成，最晚提交时间本周日（3月25日24时之前）结束，<b>逾期不接受补考,该考试分数计入平时成绩</b>
- 考试方式：请同学<font color=red><b>拷贝</b></font>该试卷至自己姓名的目录后，将文件更名为同学姓名拼音-exam7后，进行作答。例如wangwei-exam7
- 提交格式：请同学新建自己姓名全拼的文件夹，将该试卷，数据文件，zip文件等相关考试文件，放置此目录下。将该目录<b>移动</b>至/0.Teacher/Exam/7/目录下
- 注意事项：为确保同学们真正了解自身对本周课程的掌握程度，<font color=red><b>请勿翻阅，移动，更改</b></font>其它同学试卷。如发现按0分处理
- 请同学在下方同学姓名处填写自己的姓名，批改人和最终得分不用填写

- 同学姓名:<u>_洒海涛____</u>  
- 批改人： David
- 最终得分:98

<center><h1>####答卷开始####</h1></center>
<hr>

## 简答题(共6题，1，2题每题5分，后4题每题10分，共计50分)

note: 48分

#### 1. 机器学习任务中，通常会将给定数据切分为训练集，验证集，测试集，请回答这三类数据集各自的用途。

训练集: 占到绝大部分的数据, 用于训练所需要的机器学习模型.<br/>
验证集: 训练模型过程种用来检验模型是否过拟合, 评估模型效果, 调节超参数等等用的.<br/>
测试集: 用于测试最终模型的性能.

#### 2.如何将数据转换成xgboost内置用法的libsvm数据格式，以及该格式数据是如何解读的?

- note :3分  如何将数据转换成xgboost内置用法，没写~ 

该格式是稀疏向量组成的矩阵, 只标记非0值, 例如 3:1 10:1 11:1, 表示一条记录里面, 3,10,11列的值位1,其他都为0, 数据转换时按照格式标记出非0值即可.

#### 3. 对于自动特征选择，通常有哪三类方法？试写出每类方式的思想，以及在sklearn中的代码实现（面试题）

- 单特征选择.<br\>
例如子集搜索, 采用贪心法, 先选取一个最优特征$\alpha_{1}$, 第二轮再选择一个$\alpha_{2}$使得$\{\alpha_{1},\alpha_{2}\}$最优,反复直至达到所需要的特征数.

In [5]:
>>> from sklearn.datasets import load_iris
>>> from sklearn.feature_selection import SelectKBest
>>> from sklearn.feature_selection import chi2
>>> iris = load_iris()
>>> X, y = iris.data, iris.target
>>> X.shape
(150, 4)
>>> X_new = SelectKBest(chi2, k=2).fit_transform(X, y)

- 基于模型的特征选择. <br\>
先训练一个模型, 比如是一个回归模型, 那么对于每个特征都会有一个系数, 如果这个系数很小, 那么可以认为这个特征是不重要的,就可以删除这个特征. 通常会给定一个阈值, 系数小于这个阈值的特征将会被删除.

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
iris = load_iris()
X, y = iris.data, iris.target
lsvc = LinearSVC(C=0.01, penalty="l1", dual=False).fit(X, y)
model = SelectFromModel(lsvc, prefit=True)
X_new = model.transform(X)

- 递归特征消除
训练一个模型, 通过系数,重要性等选出最不重要的特征, 然后删除这个特征, 在剩下的数据集里重复执行, 直到选出需要的特征数目.

In [11]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import RFE
from sklearn.svm import LinearSVC
iris = load_iris()
X, y = iris.data, iris.target
svc = LinearSVC()
rfe = RFE(estimator=svc, n_features_to_select=2, step=1)
rfe.fit(X,y)
X_new = rfe.transform(X)

#### 4.网格搜索交叉验证的作用是什么，并简述网格搜索交叉验证是如何运行的？ 

网格搜索加上交叉验证是用来进行超参数选择的, 先通过枚举各个超参数形成组合, 然后针对每个参数, 生成一个模型, 之后通过交叉验证来评估模型性能, 最终确定最佳的超参数组合.

#### 5.工业界上模型整合有三大类方式？试简述每类方式其思想？（面试题）

1 bagging

训练多个强分类器,  最后用投票法决定模型输出, 此类模型集成可减轻模型的过拟合, 类似的算法还有随机森林.

2 boosting

训练一系列的弱分类器, 最后组成强分类器, 此类集成可以降低模型偏差, 但容易过拟合, boosting算法里面比较著名的有adboost, GBDT等

3 stacking
先学习一系列的初级学习器, 通过初级学习器产生新的数据, 在此基础上继续学习. 神经网络属于这一类算法.

#### 6.  我们可以将xgboost的中众多参数分类为哪三类？请写出哪些参数可以用什么方式用来控制过拟合？

可分为通用参数(general parameters), 增强参数(booster parameters) 和 任务参数(task parameters)

booster 参数可以用来防止过拟合:

eta :参数可以控制每一步减少权重残差的步长, 减少过拟合 <br>
gamma :可以抑制节点的分裂, 防止树太深<br>
max_depth: 限制最大树的深度<br>
min_child_weight: 观测权重之和最小值, 越小越容易过拟合<br>
lambda: L2 权重的L2正则化项,越大越不容易过拟合<br>
alpha: L1 权重的L1正则化项,越大越不容易过拟合<br>

## 实验题(共2题，每题25分，共计50分)

- note:50

### 1. 使用XGBoost内置方式，导入iris数据完成分类问题（要求以不同参数设置xgboost运行并比对），最后给出实验总结

- note: 自己写了个CV，不错。：）

In [60]:
from sklearn.datasets import load_iris
import xgboost as xgb

iris = load_iris()
X,y = iris.data, iris.target

d_train = xgb.DMatrix(data=X[:125,:], label=y[:125])
d_test = xgb.DMatrix(data=X[125:,:], label=y[125:])

watch_list = [(d_test,'eval'), (d_train,'train')]

for eta in (0.001,0.01,0.1):
    for max_dep in (1,2,3):
        print("eta",eta, "max deepth",max_dep)
        para = {"eta":eta, "max_depth":max_dep,"lambda":2,"alpha":10, 'objective':'multi:softmax', 'num_class':3};
        model = xgb.train(para, d_train, num_boost_round= 10,evals=watch_list)
        
#model.predict(X)


eta 0.001 max deepth 1
[0]	eval-merror:1	train-merror:0.2
[1]	eval-merror:1	train-merror:0.2
[2]	eval-merror:1	train-merror:0.2
[3]	eval-merror:1	train-merror:0.2
[4]	eval-merror:1	train-merror:0.2
[5]	eval-merror:1	train-merror:0.2
[6]	eval-merror:1	train-merror:0.2
[7]	eval-merror:1	train-merror:0.2
[8]	eval-merror:1	train-merror:0.2
[9]	eval-merror:1	train-merror:0.2
eta 0.001 max deepth 2
[0]	eval-merror:0.12	train-merror:0.024
[1]	eval-merror:0.12	train-merror:0.024
[2]	eval-merror:0.12	train-merror:0.024
[3]	eval-merror:0.12	train-merror:0.024
[4]	eval-merror:0.12	train-merror:0.024
[5]	eval-merror:0.12	train-merror:0.024
[6]	eval-merror:0.12	train-merror:0.024
[7]	eval-merror:0.12	train-merror:0.024
[8]	eval-merror:0.12	train-merror:0.024
[9]	eval-merror:0.12	train-merror:0.024
eta 0.001 max deepth 3
[0]	eval-merror:0.12	train-merror:0.024
[1]	eval-merror:0.12	train-merror:0.024
[2]	eval-merror:0.12	train-merror:0.024
[3]	eval-merror:0.12	train-merror:0.024
[4]	eval-merror:0.12	

结果表明 eta 过大, 树太深容易出现过拟合, 数据集太小,模型太强也会容易过拟合.

### 2. 使用XGBoost的sklearn接口，对KaggleCredit2数据完成信用卡欺诈项目的建模及分析（要求以不同参数设置xgboost运行并比对），最后给出实验总结

- KaggleCredit2数据文件 位于/home/sxy-s3/0.Teacher/Exam/KaggleCredit2.csv.zip，请勿复制或移动该文件

In [63]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)
import zipfile
with zipfile.ZipFile('/home/sxy-s3/0.Teacher/Exam/KaggleCredit2.csv.zip', 'r') as z:
    f = z.open('KaggleCredit2.csv')
    data = pd.read_csv(f, index_col=0)
data.dropna(inplace=True)
data.head()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45.0,2.0,0.802982,9120.0,13.0,0.0,6.0,0.0,2.0
1,0,0.957151,40.0,0.0,0.121876,2600.0,4.0,0.0,0.0,0.0,1.0
2,0,0.658180,38.0,1.0,0.085113,3042.0,2.0,1.0,0.0,0.0,0.0
3,0,0.233810,30.0,0.0,0.036050,3300.0,5.0,0.0,0.0,0.0,0.0
4,0,0.907239,49.0,1.0,0.024926,63588.0,7.0,0.0,1.0,0.0,0.0


In [65]:
y = data['SeriousDlqin2yrs']
X = data.drop('SeriousDlqin2yrs', axis=1)
y.mean()

0.06742876076872101

In [68]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [84]:
model = xgb.XGBClassifier( objective="binary:logistic", eta = 0.25,max_depth = 6,n_estimators=10).fit(X_train,y_train)

In [89]:
model.score(X_train,y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9382593328915396

In [86]:
model.score(X_test,y_test)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.937375745526839

In [116]:
eta= (0.05,0.1,0.3)
max_depth = (2,8,12)
min_child_weight = (25,50,75,100)
for e in eta:
    for m in max_depth:
        for c in min_child_weight:
            print("eta:{}, max_depth:{}, min_child_weight:{}".format(e,m,c))
            model = xgb.XGBClassifier(eta = e,max_depth=m, n_estimators=10 ,  min_child_weight=c, objective='binary:logistic')
            model.fit(X_train,y_train)
            print("train score:{}, test score:{}".format(model.score(X_train,y_train),model.score(X_test,y_test)))

eta:0.05, max_depth:2, min_child_weight:25


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


train score:0.9337309476474487, test score:0.9363817097415507
eta:0.05, max_depth:2, min_child_weight:50


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


train score:0.9337309476474487, test score:0.9363817097415507
eta:0.05, max_depth:2, min_child_weight:75


...

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


train score:0.9365044302088702, test score:0.9370812164052721
eta:0.3, max_depth:12, min_child_weight:75


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


train score:0.9359399160592003, test score:0.9364185258817466
eta:0.3, max_depth:12, min_child_weight:100
train score:0.9356576589843654, test score:0.9373021132464472


/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [120]:
from sklearn.model_selection import GridSearchCV

para_grid ={ "max_depth" :(3,6,9), "min_child_weight":(25,50,100)}

In [121]:
grid_search = GridSearchCV(xgb.XGBClassifier(eta = 0.3, objective='binary:logistic'), para_grid, cv = 6)

In [122]:
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this wi

GridSearchCV(cv=6, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.3, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'min_child_weight': (25, 50, 100), 'max_depth': (3, 6, 9)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [124]:
grid_search.score(X_train,y_train)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9367744152369732

In [123]:
grid_search.score(X_test,y_test)

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9370812164052721

In [125]:
grid_search.best_params_

{'max_depth': 3, 'min_child_weight': 25}

xgboost基于树模型, 不需要对特征进行缩放, 模型泛化能力好, 树的深度到达一定程度之后对模型提升有限. 由于类别不平衡,min_child_weight影响比较大

### 本阶段课程意见反馈题(非必答，不送分)
#### 请同学围绕以下两点进行回答：
- 自身总结：请您对您自己在本周课程的学习，收获，技能掌握等方面进行一次总结 ，也包括有哪些方面存在哪些不足，欠缺，困惑。作为将来回顾学习路径时的依据。


本周搜获就是知道了很多不为人知的套路

- 课程反馈：请就知识点，进度，难易度，教学方式，考试方式及难易度等方面向我们反馈，督促我们进行更有效的改进。

xgboost的原理没有讲清楚

<hr>
<center><h1>####答卷结束####</h1></center>